# **Trabajo práctico 2: modelos clasificadores basados en ensambles.** 
**Integrantes:** Rodrigo Mauriño, Martín Llada, Juan Jose Iguaran

El objetivo de este trabajo es construir modelos clasificadores basados en ensambles que permitan evaluar distintos algoritmos a a hora de predecir y analizar el otorgamiento de préstamos hipotecarios. En este sentido, se evalúa si el aplicante n, dadas sus caracetristicas -los valores de los atributos X- es elegible para el crédito. La base de datos consiste en informacion sobre créditos para vivivienda en el estado de Nueva York, EE.UU. 

Los algoritmos de aprendizaje supervisado tiendne a ser evaluados como funcionales en la medida en la que el valor predicho por el modelo sea lo mas cercano posible al valor real. De esta manera, lo acertado de la prediccion se ve afectado principalmente por tres variables: sesgo, varianza y error. el sesgo es lo alejado que esta la prediccion del valor real; la varianza representa cuan dispersas estan estas predicciones y el error, es una medida de el porcentaje que es irreductible para un modelo.

Para lidiar con estos problemas, se cuentan con los metodos de ensambles, que como su nombre lo indica, son ensambles de algoritmos de aprendizaje automatico que permiten reducir lo reductible en una prediccion, es decir, el sesgo y la varianza. 

Para reducir el sesgo se cuenta con diversas tecnicas dentro de las que destaca el boosting, en donde se utiliza el resultadode una prediccion de un modelo como entrada para realizar la siguiente prediccio, de esta manera aumentando la precision del modelo y por consiguiente reduciendo el sesgo. Dentro de los algoritmos de boosting se encuentra el XGBOOST.

En cuanto a se refiere a la varianza, se cuentan con tecnicas como el bagging, en donde el procedimiento consiste en generar multiples fuentes de datos y entrenar a partir de estos distintos modelos y utilizar estos resultados una vez ponderados para el resultado final. Dentro de los algoritmos de bagging se encuentra tanto random forest, como el algoritmo homonimo.

El objetivo del presente trabajo consite en aplicar los distintos metodos de ensambles a arboles decesion con el objetivo de aumentar el poder clasificatorio de la tecnica para el presente caso de creditos hipotecarios en el estado de Nueva York

# Importación de librerías y base de datos

En primer lugar, y en pos de entrenar los modelos clasificatorios, es necesario alistar el framework mediante la importacion de las librerías necesarias:

In [73]:
import pandas as pd 
import numpy as np
import sklearn as sk
import random
from sklearn import model_selection
from sklearn import metrics
from xgboost import XGBClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_validate

Una vez importadas las librerías, es necesario importar el dataset a ser utlizado, asi como una transformacion de la variable objetivo para reducirla a un termino binario el cual es el apto para los modelos a utilizar:

In [3]:
data = pd.read_csv("baseclean.csv")
dataf=data.copy()
dataf=dataf.dropna()
values = np.array(dataf["county_name"])
integer_encoded = sk.preprocessing.LabelEncoder().fit_transform(values)
dataf.loc[:,"county_name"]=integer_encoded
lista = [1,5,6]
target_encoded =dataf['action_taken'].isin(lista).astype(int)

Se particionan los datos en test y desarrollo

In [4]:
X=dataf.iloc[:,1:]
Y=target_encoded
dev_x, test_x, dev_y, test_y = model_selection.train_test_split(X,Y,train_size=0.8, test_size=0.2,random_state=123)

# Boosting

Se procede a implementar el algoritmo XGBOOST, el cual es un algoritmo de boosting. Para ello se establecen los hiperparametros del modelo.

In [122]:
gb = XGBClassifier(max_depth=15,
                           min_child_weight=1,
                           learning_rate=.5,
                           n_estimators=250,
                           silent =1,
                           objective='binary:logistic',
                           gamma=0,
                           max_delta_step=0,
                           subsample=1,
                           colsample_bytree=1,
                           colsample_bylevel=1,
                           reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=30,
                           seed=1)

Una vez creado el modelo se procede a entrenarlo utilizando los datos de desarrollo.

In [123]:
gb.fit(dev_x, dev_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.5,
       max_delta_step=0, max_depth=15, min_child_weight=1, missing=None,
       n_estimators=250, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=0, scale_pos_weight=30, seed=1, silent=1, subsample=1,
       verbosity=1)

Una vez entrenado el modelo, se procede a evaluar su desempeño a paritr de las metricas accuracy, precsion, recall, f1_score y la curva roc_auc

In [124]:
a=model_selection.cross_val_score(gb,dev_x,dev_y,cv=5)
predictions=gb.predict(test_x)
acc=metrics.accuracy_score(test_y, predictions)
pre=metrics.precision_score(test_y, predictions)
rec=metrics.recall_score(test_y, predictions)
fs=metrics.f1_score(test_y, predictions)
roc=metrics.roc_auc_score(test_y,predictions)

#c=confusion_matrix(test_y, predictions)

In [125]:
print(a)
print(acc,pre,rec,fs,roc)

[0.79569892 0.79856115 0.80755396 0.80215827 0.79316547]
0.8204022988505747 0.86 0.8865979381443299 0.8730964467005077 0.7774221918209802


Se observa que todas las mtreicas a excepcion de la curva roc_auc, muestran valores superiores al 80%, lo que muestra su superioridad con respecto a los arboles tradicionales. A continuacion se procede a aplicar el algoritmo GridSearch para buscar los mejores hiperparametros del modelo

In [67]:
gb = XGBClassifier(objective='binary:logistic')

clf = model_selection.GridSearchCV(gb,{'max_depth': np.arange(1, 30),}, 
                        verbose=1, 
                        error_score='neg_log_loss')

clf.fit(dev_x,dev_y)
clf.best_score_, clf.best_params_

/home/juan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 29 candidates, totalling 87 fits


[Parallel(n_jobs=1)]: Done  87 out of  87 | elapsed:  1.1min finished


(0.8339324227174695, {'max_depth': 2})

Se encontro que los mejores hiperparametros fue una profundidad maxima de 2, por lo que se procede a crear un modelo con dichas caracteristicas.

In [126]:
gb = XGBClassifier(max_depth=2,learning_rate=.1,n_estimators=100)

Se procede a entrenar el modelo creado con los datos de desarrollo.

In [127]:
gb.fit(dev_x, dev_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

Una vez entrenado el modelo, se calculan las mestricas correspondientes

In [128]:
a=model_selection.cross_val_score(gb,dev_x,dev_y,cv=5)
predictions=gb.predict(test_x)
acc=metrics.accuracy_score(test_y, predictions)
pre=metrics.precision_score(test_y, predictions)
rec=metrics.recall_score(test_y, predictions)
fs=metrics.f1_score(test_y, predictions)
roc=metrics.roc_auc_score(test_y,predictions)

#c=confusion_matrix(test_y, predictions)
print(a)
print(acc,pre,rec,fs,roc)

[0.82437276 0.82553957 0.83273381 0.84172662 0.8147482 ]
0.8060344827586207 0.8787878787878788 0.8371134020618557 0.8574445617740233 0.78585527922998


Se puede apreicar que todas las metricas de desempeño mejoran una vez se han utilizado los hipermarametros encontrados. AHora se procede a evaluar otro modelo de ensamble, mas concretamente, Bagging

# Bagging

Al igual que con boosting, el primer paso es crear el objeto de bagging, utilizando como modelo los arboles de decision. Asi mimo se establecera como 100 el numero de estimadores que es el valor default.

In [108]:
bagg = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, random_state=7)

Una vez creado el modelo se procede a entrenarlo

In [109]:
bagg.fit(dev_x, dev_y)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=100, n_jobs=None, oob_score=False,
         random_state=7, verbose=0, warm_start=False)

Al entrenar el parametro se procede a evaluarlo a partir de las metricas previamente mencionadas.

In [129]:
a=model_selection.cross_val_score(bagg,dev_x,dev_y,cv=5)
predictions=bagg.predict(test_x)
acc=metrics.accuracy_score(test_y, predictions)
pre=metrics.precision_score(test_y, predictions)
rec=metrics.recall_score(test_y, predictions)
fs=metrics.f1_score(test_y, predictions)
roc=metrics.roc_auc_score(test_y,predictions)
print(a)
print(acc,pre,rec,fs,roc)

[0.81182796 0.81834532 0.82014388 0.82194245 0.82194245]
0.8218390804597702 0.8898488120950324 0.8494845360824742 0.869198312236287 0.8038891874725168


Se observa que al igual que con boosting, todas las metricas se encuentran alrededor 80%, estando estas un poco por encima lo que indicaria que el problema del arbol de clasificacion esta mas ligado a la varianza que al sesgo. A continuacion se procede a estimar a partir de gridsearch el mejor numero de estimadores

In [88]:
bagg = BaggingClassifier()
param = {'n_estimators': np.arange(1, 200)}
tree = sk.model_selection.GridSearchCV(bagg, param, cv= 5)
tree.fit(dev_x, dev_y)
tree.best_params_

{'n_estimators': 126}

Se encontro que el mejor numero d eestimadores, son 126, por lo que se creara un modelo a partir de esta informacion.

In [115]:
bagg2 = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=126, random_state=7)

Se procede entrenar el modelo

In [116]:
bagg2.fit(dev_x, dev_y)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=126, n_jobs=None, oob_score=False,
         random_state=7, verbose=0, warm_start=False)

Una vez entrenado el modelo se procede a la evaluacion de las mestricas

In [118]:
a=model_selection.cross_val_score(bagg2,dev_x,dev_y,cv=5)
predictions=bagg2.predict(test_x)
acc=metrics.accuracy_score(test_y, predictions)
pre=metrics.precision_score(test_y, predictions)
rec=metrics.recall_score(test_y, predictions)
fs=metrics.f1_score(test_y, predictions)
roc=metrics.roc_auc_score(test_y,predictions)
print(a)
print(acc,pre,rec,fs,roc)

[0.81003584 0.82374101 0.82374101 0.82553957 0.82733813]
0.8160919540229885 0.8838709677419355 0.8474226804123711 0.8652631578947368 0.7957492548981288


Se observa que los valores aun cuando semejantes,se encuentran por debajo de los obtenidos anteriormente por lo que se concluye que los valores dfaoult del algoritmo son lo suficientemente buenos. A continuacion se procedera con otra tecnica de bagging, random forest.

# Random Forest

Se procede a crear un objeto tipo RandomForestClassifier con altura máxima altura 3, utilizando 800 arboles (con 400 es suficiente para obtener una buena perfomance) y manteniendo el resto de los hiperparámetros en su valor preestablecido:

In [16]:
dtree =RandomForestClassifier(n_estimators=800, max_depth=3, random_state=0)

In [10]:
dtree

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

Una vez creado el objeto, se procede a estimar el desempeño del modelo utilizando 5-fold cross validation a traves de la función  cross_val_nuevo utilizando la medida de accuracy:

In [18]:
perf_3tree = cross_validate(dtree,dev_x, dev_y, cv = 5)

In [41]:
perf_3tree

{'fit_time': array([1.34262609, 1.28496718, 1.28660011, 1.30285692, 1.28607297]),
 'score_time': array([0.0935998 , 0.08285928, 0.08324218, 0.08415532, 0.08428788]),
 'test_score': array([0.73476703, 0.74280576, 0.75      , 0.73021583, 0.74460432]),
 'train_score': array([0.76034173, 0.75965858, 0.75112309, 0.75651393, 0.75471698])}

In [27]:
dtree.fit(dev_x, dev_y)
predictions=dtree.predict(test_x)
acc=metrics.accuracy_score(test_y, predictions)
pre=metrics.precision_score(test_y, predictions)
rec=metrics.recall_score(test_y, predictions)
fs=metrics.f1_score(test_y, predictions)
roc=metrics.roc_auc_score(test_y,predictions)  

In [28]:
print("Accuracy:",acc)
print("Precisión:",pre)
print("Recall:",rec)
print("F1_score:",fs)

Accuracy: 0.7413793103448276
Precisión: 0.7447833065810594
Recall: 0.9567010309278351
F1_score: 0.8375451263537907


A partir del árbol entrenado, se computaron las métricas accucary (es decir, la proporción de las observación pronosticada correctamente con respecto al total de observaciones), precisión (la relación entre las observaciones pronosticadas positivas correctamente y el total de observaciones pronosticadas positivas), recall (la relacion entre las obvevaciones pronosticadas correctamente y el total de las observaciones asociadas a la clase positiva) y F1_score (el cual es un promedio artimético de las métricas recall y precisión) utilizando la matriz de confusión que se deriva del set de datos de validación. Se puede apreciar que para el árbol entrenado utilizando Random Forest, la accuracy  es del 74%, mientras que la precision y el recall  se ubican en 74% (un nivel considerables que se asocia baja tasa de falsos positivos) y 96%, respectivamente. En este caso de estudio, la métrica de precisión es elevada indicando que el modelo posee relativamente pocos casos en los cuales otorga un crédito a personas que no poseen las condiciones para recibirlo (esto se asocia con potenciales incovenientes en el recobro de estos crédito) y, a su vez, posee una elevada tasa de recall indicando que el modelo posee relativamente bajos registros para los cuales no asigna créditos a personas que sí cumples con las condiciones de acceder al mismo (esto se asocia con potenciales pérdidas de clientes). Si bien ambas métricas son relevantes, teniendo en cuenta el caso de estudio, se considera que la infromacion más relavante la captura el indicador recall. Por ultimo, teniendo en cuenta que ambas indicadores son relevante para el caso de estudio, es conveniente analizar una métrica que tiene en cuenta tanto los falsos positivos como los falsos negativos: F1-score, el cual se ubica en el orden del 84%. Asi mismo, se evidencia que para los datos de entrenamiento se tiene  medidas relativamente parecidas de performance. 

A continuación se calcularan las medias y los desvios estandar para todos las medidas de performance, para ello, primero se transformaran las listas en un arreglo de numpy:


In [55]:
train_acc = perf_3tree['train_score'][1]
val_acc = perf_3tree['test_score'][1]
print("Accuracy media_train:",round(train_acc.mean(),3))
print("Accuracy media_val:",round(val_acc.mean(),3))

Accuracy media_train: 0.76
Accuracy media_val: 0.743


/home/juan/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


Se puede apreciar que para ambas medidas, la media es mas alta para los datos de entrenamiento respecto de las mismas metricas que se observan para los datos de validación, lo que indica es que el modelo se comporta mejor para los datos con los que fue entrenado.

Ahora, se pocederá a realizar a entrenar el modelo de mejor desempeño en cada caso utilizando el set de entrenamiento, luego se realizara su evualuación y comparación con los resultados obtenidos y exhibidos previamente. En el caso de arboles de decisión, para encontrar los parámetro de dicho árbol se empleara GridSearchCV:

In [82]:
param = {'max_depth': np.arange(1, 30),'criterion' : ('entropy','gini'), }
tree = sk.model_selection.GridSearchCV(RandomForestClassifier(n_estimators=100), param, cv= 5)
tree.fit(dev_x, dev_y)
tree.best_params_

{'criterion': 'gini', 'max_depth': 24}

Los resultados permite apreciar que la mejor profundidad es 16 y el mejor método es gini. Por lo tanto, ahora se utilizará este modelo y el árbol con los hiperparametros encontrados para realizar las validaciones correspondientes utilizando accucary como metrica, asi como calcular las medias y los desvíos de los resultados para volverlos comparables:

In [56]:
dtree2 =RandomForestClassifier(n_estimators=100, max_depth=16, random_state=0, criterion='gini')

In [57]:
perf_3tree_best = cross_validate(dtree2,dev_x, dev_y, cv = 5)

In [58]:
dtree2.fit(dev_x, dev_y)
predictions=dtree2.predict(test_x)
acc=metrics.accuracy_score(test_y, predictions)
pre=metrics.precision_score(test_y, predictions)
rec=metrics.recall_score(test_y, predictions)
fs=metrics.f1_score(test_y, predictions)
roc=metrics.roc_auc_score(test_y,predictions) 

In [59]:
print("Accuracy:",acc)
print("Precisión:",pre)
print("Recall:",rec)
print("F1_score:",fs)

Accuracy: 0.8103448275862069
Precisión: 0.870020964360587
Recall: 0.8556701030927835
F1_score: 0.8627858627858628


In [60]:
train_acc = perf_3tree_best['train_score'][1]
val_acc = perf_3tree_best['test_score'][1]
print("Accuracy media_train:",round(train_acc.mean(),3))
print("Accuracy desvío_train:",round(train_acc.std(),3))
print("Accuracy media_val:",round(val_acc.mean(),3))
print("Accuracy desvío_val:",round(val_acc.std(),3))

Accuracy media_train: 1.0
Accuracy desvío_train: 0.0
Accuracy media_val: 0.806
Accuracy desvío_val: 0.0


/home/juan/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


Por lo tanto, se puede apreciar que todas las metricas mejoraron (accuracy, precision, recall y F1-score) respecto aquellas exhibidas previamente. A su vez, la accuracy promedio mejoro ubicandose en el orden del 81% (vs 74%).